In [1]:
from SAC.Normal_Policy import Normal_Policy
from SAC.Value_function import Value_function
from SAC.SoftActorCritc import SoftActorCritic, plot

import time
import gym
import numpy as np

import sys
sys.path.insert(0, './../../laser-hockey-env')
import laser_hockey_env as lh
from importlib import reload

import progressbar
from tensorboardX import SummaryWriter
import tensorflow as tf


In [2]:
env_name = "Laser_hockey"
env = lh.LaserHockeyEnv()

In [3]:
ac_space = env.action_space
o_space = env.observation_space
print(ac_space.low)
print(o_space)

[-1. -1. -1. -1. -1. -1.]
Box(16,)


In [4]:
ac_space.shape

(6,)

In [5]:
direct = "C:/Users/Maximilian/Documents/Studium/Informatik/3. Semester/Intelligent Systems/project/DeepRLHockey/code/SAC/weights/"
version = "v_full_16"
save_path_new = direct+version

In [6]:
q_fct_config = {"hidden_layers": [256, 256]}
v_fct_config = {"hidden_layers": [256, 256]}
pi_fct_config = {"hidden_layers": [256, 256], "dim":3}#, "output_act_fct_mu":tf.tanh}

SAC_agent = SoftActorCritic(o_space=o_space, 
                            a_space=ac_space, 
                            value_fct=Value_function, 
                            policy_fct=Normal_Policy, 
                            env= env,
                            discount=  0.91, 
                            tau = 0.005, 
                            batch_size=256, 
                            alpha = 'auto',
                            lambda_V = 3e-4, 
                            lambda_Pi = 3e-4, 
                            lambda_Q = 3e-4,
                            dim_act = 3,
                            dim_obs = 16,
                            buffer_size = int(1e6),
                            q_fct_config=q_fct_config,
                            v_fct_config=v_fct_config,
                            pi_fct_config=pi_fct_config, 
                            save_path = save_path_new) 

In [7]:
player2 = lh.BasicOpponent()

In [8]:
writer = SummaryWriter('visualization/{}-{}'.format(env_name, version))

In [9]:

def train(iter_fit=1000, max_steps= 500,  env_steps = 1, grad_steps = 1, burn_in =5, writer = None):
        start = SAC_agent._sess.run(SAC_agent.global_step)
        saver = SAC_agent._saver
        bar = progressbar.ProgressBar(max_value=iter_fit)
        # Init Statistics      
        winner_count = 0
        draw_count = 0
        total = start
        
        j = start
        for i in range(iter_fit):  
            ob = env.reset()     
            obs_agent2 = env.obs_agent_two()
            
            total_reward = 0
            
            for _ in range(max_steps):
                for e_i in range(env_steps):#
                    
                    if j < burn_in:
                        a = env.action_space.sample()
                        a = a[:3]                     
                    else:
                        a = SAC_agent.action(np.asarray(ob).reshape(1, SAC_agent._config["dim_obs"]))
                        a = a[0]
                        
                    a2 = player2.act(obs_agent2)                    
                    comb_a = np.hstack([a,a2])
                    (ob_new, reward, done, _info) = env.step(comb_a)
                     
                    
                    reward = reward - (_info['reward_puck_direction'] == 0.0) * 0.02
                    total_reward += reward
                    
                    SAC_agent.buffer.store(ob, a, reward, ob_new, done)
                    ob=ob_new
                    obs_agent2 = env.obs_agent_two()
  
                if j  >= 5:
                    for g_i in range(grad_steps):
                        update_value_target = False
                        if i % SAC_agent._config["target_update"] == 0:
                            update_value_target = True
                        loss_V_fct, loss_Q1_fct,loss_Q2_fct,loss_PI_fct= SAC_agent._train(update_value_target)
 
                if done:
                    break 

            j += 1
            winner_count +=  _info['winner'] == 1
            draw_count += _info['winner'] == 0
            total += 1              
            SAC_agent._sess.run(SAC_agent.increment_global_step_op)

            if i>0 and i % 250 == 0:
                saver.save(SAC_agent._sess, save_path_new)
            if writer: 
                writer.add_scalar('win_loss_ratio/wins',winner_count/total , start+i)
                writer.add_scalar('win_loss_ratio/draws',draw_count/total , start+i)
                writer.add_scalar('reward',total_reward , start+i)
            bar.update(i)   
        return winner_count

In [10]:
train(10000, burn_in = 0, writer = writer)

 53% (5353 of 10000) |#########         | Elapsed Time: 10:05:36 ETA:  12:39:58

KeyboardInterrupt: 

In [11]:
SAC_agent._saver.save(SAC_agent._sess, save_path_new)

'C:/Users/Maximilian/Documents/Studium/Informatik/3. Semester/Intelligent Systems/project/DeepRLHockey/code/SAC/weights/v_full_16'

In [12]:
SAC_agent._sess.run([SAC_agent.alpha, SAC_agent.log_alpha])

[0.0, -570.86664]

In [13]:

for i in range(10):
    ob = env.reset()
    for _ in range(1000):
        env.render()
#        distance_to_opponent = np.sqrt((ob[6] -ob[0])**2 + (ob[7] -ob[1])**2)
 #       distance_to_puck = np.sqrt((ob[12] -ob[0])**2 + (ob[13] -ob[1])**2)
  #      angle_to_opponent = np.arctan2((ob[7] -ob[1]),(ob[6] -ob[0])) - ob[2]
   #     angle_to_puck = np.arctan2((ob[13] -ob[1]),(ob[12] -ob[0])) - ob[2]
    #    ob = np.append(ob, [distance_to_opponent, distance_to_puck,angle_to_opponent, angle_to_puck] )
        obs_agent2 = env.obs_agent_two()    
        #a = SAC_agent.act_greedy(np.asarray(ob).reshape(1, SAC_agent._config["dim_obs"]))
        fddct = {SAC_agent.obs : np.asarray(ob).reshape(1, SAC_agent._config["dim_obs"])}
        #actions = self._sess.run([tf.tanh(self._Policy.mu, name = "tanh")], feed_dict=fddct)
        a1 = SAC_agent._sess.run([SAC_agent._Policy.mu], feed_dict=fddct)[0][0]
        print(a1)
        #print(a1)
        #print(a1, SAC_agent.act_greedy(np.asarray(ob).reshape(1, SAC_agent._config["dim_obs"]))[0])
        #a1 = player2.act(ob)
        
        a2 = player2.act(obs_agent2)
        
        ob, r, d, info = env.step(np.hstack([a1,a2]))    
        #print(info)
        obs_agent2 = env.obs_agent_two()
        if d: break
env.close()

[ 0.06662476  0.39223385 -0.33989695]
[-0.08694412  0.03304577 -0.17113331]
[-0.08613198 -0.21390551  0.1842733 ]
[-0.01999039 -0.23713285  0.29389912]
[ 0.2874176 -0.3391828  0.5567144]
[ 0.47654808 -0.22942227  0.48397353]
[0.6260791  0.03132564 0.15987977]
[ 0.63347375  0.2889474  -0.11341056]
[ 0.6293023   0.4381562  -0.31183687]
[ 0.55101967  0.3582573  -0.29309443]
[ 0.44743246  0.24453485 -0.27506682]
[ 0.35272163  0.19974107 -0.16141954]
[ 0.21105857  0.165371   -0.14714411]
[ 0.06535773  0.07934076 -0.15221915]
[ 0.01036398  0.02922547 -0.12674657]
[ 0.02618776 -0.01587945 -0.0850015 ]
[ 0.08322845 -0.05736548 -0.03638187]
[ 0.07242171 -0.08414334 -0.03618124]
[ 0.06058256 -0.14811522 -0.0801104 ]
[ 0.07409789 -0.02559072 -0.01956657]
[ 0.04307775 -0.04675049 -0.01879367]
[ 0.05866061 -0.11254114 -0.07161739]
[ 0.05425613 -0.12575537 -0.06615028]
[ 0.05169569 -0.14297587 -0.06438133]
[0.00240003 0.01347548 0.04177031]
[ 0.0317186  -0.04463762 -0.01620194]
[ 0.03104852 -0.06816

[ 0.00718395 -0.02038151  0.02870998]
[ 0.02218817 -0.05419809 -0.00156036]
[ 0.01862304 -0.04585463  0.00567761]
[ 0.01912688 -0.04920799  0.00433353]
[ 0.02069364 -0.05628103  0.0006493 ]
[ 0.0185899  -0.06198388  0.00335059]
[ 0.01190375 -0.02787358  0.01965275]
[ 0.00803117 -0.02366728  0.02679738]
[ 0.00948529 -0.03292006  0.02298102]
[ 0.00472371 -0.0325287   0.03105965]
[ 0.00210397 -0.00812674  0.03907761]
[ 0.00371347 -0.02203304  0.03431204]
[ 0.00097381 -0.03530949  0.03696564]
[ 0.00056349 -0.02244776  0.03955689]
[ 0.00046246 -0.039437    0.03721717]
[-0.00031222 -0.03798753  0.03873625]
[ 0.0012518  -0.03972989  0.03584436]
[-0.00018509 -0.03276092  0.03929415]
[-0.00053532 -0.03200942  0.03999522]
[-0.00092359 -0.03635949  0.04000655]
[ 0.00051396 -0.04096454  0.03690472]
[-0.00328882 -0.01587707  0.04701689]
[-0.00547399 -0.00949413  0.05164066]
[-0.0040396  -0.02174085  0.04741529]
[-0.0112703  -0.02151662  0.0596284 ]
[-0.00831802 -0.01425165  0.0557287 ]
[-0.00614138

[0.02424859 0.00782144 0.00413117]
[0.00617008 0.01711434 0.03595737]
[ 0.02636532 -0.00128824 -0.00078019]
[0.00327782 0.01977664 0.04122266]
[0.00464974 0.02709061 0.03999206]
[0.01288931 0.02758819 0.02618614]
[-0.00068988  0.02084863  0.04806456]
[ 0.02404819 -0.00422031  0.00268981]
[-0.00150098  0.02216798  0.04962584]
[0.02012055 0.00600487 0.0108166 ]
[-0.00563462  0.02341038  0.05677238]
[-0.00420518  0.01989764  0.05384567]
[ 0.02526622 -0.00497228  0.00052699]
[-0.00874408  0.0280202   0.06269142]
[ 0.02125762 -0.00242627  0.00765565]
[-0.00851668  0.02933556  0.06250259]
[0.01794378 0.00693733 0.01462099]
[-0.0103686   0.02446103  0.0649021 ]
[-0.01064308  0.02984071  0.06615916]
[0.02013908 0.0018785  0.01017591]
[-0.01390459  0.03423536  0.07230231]
[0.01854746 0.00090474 0.01271299]
[-0.0004002   0.01365489  0.04651383]
[0.01995139 0.00086182 0.01034167]
[-0.00258161  0.01553375  0.05046597]
[0.01813196 0.00201231 0.0135766 ]
[-0.01225163  0.03486764  0.0696114 ]
[-0.009

[ 0.01878856 -0.07019573  0.00180283]
[ 0.01303725 -0.08596045  0.00916174]
[ 0.0287609  -0.01458174 -0.00677946]
[ 0.04066305 -0.05131251 -0.03225502]
[ 0.03693466 -0.06942612 -0.02865055]
[ 0.03252171 -0.07158202 -0.02153543]
[ 0.0198351  -0.06903356  0.00021163]
[ 0.02001499 -0.05851763  0.00146219]
[ 0.01787816 -0.06428832  0.00420907]
[ 0.02649681 -0.03097087 -0.00538674]
[ 0.02944337 -0.05752879 -0.01427385]
[ 0.0372936  -0.05734974 -0.02747104]
[ 0.02675258 -0.05610007 -0.0095301 ]
[ 0.01915668 -0.05621964  0.00324747]
[ 0.01257186 -0.06579083  0.01292554]
[0.00787704 0.00715691 0.03161088]
[0.0195642  0.00282717 0.01128426]
[ 0.02788429 -0.02904719 -0.00743976]
[ 0.03840177 -0.04907554 -0.02811542]
[ 0.0357555  -0.05673605 -0.02478936]
[ 0.02460544 -0.05985421 -0.00646785]
[0.01039355 0.01353025 0.02831349]
[ 0.03613184 -0.01154929 -0.0187476 ]
[ 0.03646903 -0.04881722 -0.02482149]
[ 0.03870808 -0.06522816 -0.03101775]
[0.01663117 0.00613183 0.01671329]
[ 0.03865604 -0.02750307

KeyboardInterrupt: 

In [14]:
env.close()